# Example to make a Rotating Globe

Let's have some fun by trying to make a rotating globe, display some locations fading in and out with time, maybe even add the day/night shadows for more fun and ... Well, I just hope that it looks somewhat nice at the end when using python routines only (otherwise I might need to do some more advanced image magic or even worse 3D modelling). 

## Globe Animation (with cartopy based on Natural Earth data)

As a baseline, I've used the following answer on stackoverflow to produce a quick animation: 
https://stackoverflow.com/questions/51801109/animate-a-point-between-two-points-along-with-rotating-earth

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs

import numpy as np

import matplotlib.animation as animation
import matplotlib.pyplot as plt
%matplotlib notebook

plt.figure(figsize=(4,4))

def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

    
def animate(i):
    lon = i

    ax = plt.gca()
    ax.remove()

    ax = plt.axes([0, 0, 1, 1], projection=ccrs.Orthographic(
        central_latitude=0, central_longitude=lon))
    decorate_axes(ax)


ani = animation.FuncAnimation(
    plt.gcf(), animate,
    frames=np.linspace(360, 0, 60),
    interval=125, repeat=False)

plt.show()

## Adding Geographical Locations

We want to show a few locations on the map. So we use the geo-locations of the participants of the International Cosmic Day (https://icd.desy.de/). A day, where students, teachers and scientists from all around the globe come together to learn about cosmic rays. 

In [ ]:
import geopandas as gpd

# read in geo location from google maps export
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
fp = "/home/prokoph/DESY/outreach/ICD2021/Participants2020.kml"
df = gpd.read_file(fp, driver='KML')

In [ ]:
plt.figure(figsize=(7,7))

def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

# Define the CartoPy CRS object. This one will change during the animation. So let's keep it more general here.
globe_proj = ccrs.Orthographic(central_latitude=23, central_longitude=60)

# Now, we need to convert our GeoPandas data points into the same crs as the cartopy globe projection. 
# So we convert cartopy crs into a `proj4` string/dict compatible with GeoPandas and apply it to the data.
crs_proj4 = globe_proj.proj4_init
df_orto = df.to_crs(crs_proj4)

ax = plt.axes([0, 0, 1, 1], projection=globe_proj)
df_orto.plot(ax=ax,edgecolor='None',facecolor='red', alpha=0.5, marker='o')
decorate_axes(ax)


In [ ]:
plt.figure(figsize=(4,4))

def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

    
def animate(i):
    lon = i

    ax = plt.gca()
    ax.remove()

    # Define the CartoPy CRS object.
    globe_proj = ccrs.Orthographic(central_latitude=23, central_longitude=lon)

    # convert geolocations into a `proj4` string/dict compatible with GeoPandas
    crs_proj4 = globe_proj.proj4_init
    df_orto = df.to_crs(crs_proj4)

    ax = plt.axes([0, 0, 1, 1], projection=globe_proj)
    df_orto.plot(ax=ax,edgecolor='None',facecolor='red', alpha=0.5, marker='o')
    
    decorate_axes(ax)


ani = animation.FuncAnimation(
    plt.gcf(), animate,
    frames=np.linspace(180,-180, 60),
    interval=125, repeat=False)

plt.show()
# ani.save('../images/globe.gif', writer='imagemagick', dpi=plt.gcf().dpi)